## Proposal benchmark model

In [32]:
import torch # install using $ pip install torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import os

## Dataset preprocessing

In [19]:
# Set dataset locations
emissions_file = "01_Data/01_Carbon_emissions/AirNow/World_locations_2020_avg.csv"
selected_output_variable = "NO2"
satellite_file = ""

In [9]:
def process_images(image_files):
    """
    EDIT LATER
    Load and reshape images into an array where each column represents one image
    Input:
        filepath to folder with images or image filename if already aggregated
    Output:
        tensor of size (resolution * resolution * num_channelss, num_images)
    """
    data = np.random((res*res*num_channels, num_images)) # fake data for now
    x_train = torch.tensor(data)
    return x_train

In [26]:
def process_emissions(emissions_file, selected_output_variable):
    """
    Load emission measurements into one column
    Input:
        path to local folder that contains emission file
        selected output variable indicating measurement to be predicted
    Output:
        data frame of size (number of examples, 3) including lat, lon and output
    """
    emissions_data = pd.read_csv(emissions_file)
    
    # Filter selected output variable
    emissions_data = emissions_data[emissions_data['type'] == selected_output_variable]
    
    # Distinguish between preprocessing for classification and regression
    if "AQI" in selected_output_variable:
        emissions_data['output'] = if emissions_data['value'] <= 50 'good' elif 
    else:
        emissions_data['output'] = emissions_data['values']
    
    #emissions_labels = torch.randint(num_categories,(num_images,1)) # fake data
    #emissions = nn.functional.one_hot(emissions_labels, num_categories)
    return emissions[['lat', 'lon', 'output']]

SyntaxError: invalid syntax (<ipython-input-26-18880dbee0d2>, line 18)

In [33]:
def map_emissions_to_images():
    pass

## Train model

In [ ]:
# load data
X = process_images(satellite_file)
print("size of X: ", X.shape)
y = process_emissions(emissions_file, selected_output_variable)
print("size of Y: ", y.shape)

# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train model
if "AQI" in selected_output_variable:
    model = MLPClassifier(random_state=1, max_iter=500).fit(X_train, y_train)
else:
    model = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)

# Evaluate performance
model.score(X_test, y_test)

In [6]:
res = 64
num_channels = 3 # will be 7 later
num_images = 10
learning_rate = 0.05
num_categories = 6
classes = ['good', 'moderate', 'unhealthy_sensitive_groups',
           'unealthy', 'very_unhealthy', 'hazardous']

In [124]:
class Net(nn.Module):
    """
    Define the neural network
    """
    def __init__(self):
        super(Net,self).__init__()
        self.layer1 = nn.Linear(res*res*num_channels, num_categories) # One linear layer

    def forward(self, x):
        """
        Define forward pass
        """
        x = self.layer1(x)
        # x = nn.softmax(x)  # Softmax activation
        x = nn.functional.softmax(x, dim=1)
        return x

In [128]:
def train(x_train, y_train):
    """
    Train model on training data
    Input:
        x data and labels
    Output:
        trained model
    """
    # model = torchvision.models.resnet18(pretrained=True)
    # loss_sum = 0
    
    for epoch in range(1):
        optimizer.zero_grad() # zero gradients
        x_train = x_train.reshape(num_images, res*res*num_channels)
        # x_train = x_train.to(torch.float64)
        output = net(x_train.float())
        print(output)
        print(y_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        # loss_sum += loss.item()  

    # save trained model
    # PATH = ('./models/benchmark_model.pth')
    # torch.save(net.state_dict(), PATH)
    return net

In [31]:
# Main

net = Net()
criterion = nn.NLLLoss() # nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate)

# load data
x_train = process_images("01_")
print("size of x_train: ", x_train.shape)
y_train = process_emissions("foo.csv")
print(y_train)
print("size of y_train: ", y_train.shape)

# Train model
print(net)
train(x_train, y_train)

NameError: name 'res' is not defined